# CryoET Quality Control & Benchmarking Tool

## Overview

Inspired by [PixelPatrol](https://helmholtz-imaging.de/news/pixelpatrol-scientific-image-quality-control/), this tool provides:

### Quality Control Features (PixelPatrol-style):
- 📊 **Dataset profiling**: Metadata and pixel distribution analysis
- 🔍 **Outlier detection**: Identify anomalies and inconsistencies
- 📈 **Statistical validation**: Comprehensive data integrity checks
- 🎯 **Visual inspection**: Interactive dashboards and plots

### Benchmarking Integration:
- 🗜️ **Compression testing**: Multiple codec configurations
- ✅ **Data integrity**: Verify lossless compression
- ⚡ **Performance metrics**: Speed and throughput analysis
- 💾 **Storage optimization**: Best compression recommendations

### Combined Analysis:
- 🔬 **Pre-compression QC**: Validate data quality before compression
- 🧪 **Post-compression QC**: Ensure no data corruption
- 📋 **Automated reports**: Comprehensive QC + benchmark dashboards
- ✨ **Smart recommendations**: Data-driven optimization suggestions

---

## Workflow

```
1. Load Data → 2. Pre-QC → 3. Benchmark → 4. Post-QC → 5. Report
```

This ensures:
- Data is valid before compression
- Compression is lossless
- Performance is optimal
- Results are reproducible

In [ ]:
# Imports
import json
import pathlib
import time
import warnings
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import seaborn as sns
import zarr
from cryoet_data_portal import Client, Dataset
from scipy import stats
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

from zarr_benchmarks import utils
from zarr_benchmarks.read_write_zarr import read_write_zarr

warnings.filterwarnings("ignore")

# Visualization settings
sns.set_style("whitegrid")
plt.rcParams["figure.dpi"] = 100

print("✓ All imports successful")

## Configuration

In [ ]:
# Data configuration
DOWNLOAD_SIZE = 256
DATASET_ID = 10445

# QC thresholds
QC_THRESHOLDS = {
    "outlier_std": 3.0,  # Standard deviations for outlier detection
    "min_snr": 1.0,  # Minimum signal-to-noise ratio
    "max_saturation": 0.01,  # Maximum fraction of saturated pixels
    "min_dynamic_range": 100,  # Minimum intensity range
}

# Benchmark configuration
BENCHMARK_CODECS = [
    ("blosc_zstd", "shuffle", 3),
    ("blosc_zstd", "shuffle", 5),
    ("blosc_lz4", "shuffle", 5),
    ("zstd", None, 5),
]

CHUNK_SIZE = 128
ZARR_SPEC = 2

# Output directory
OUTPUT_DIR = pathlib.Path("data/output/qc_benchmark")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Configuration:")
print(f"  Download size: {DOWNLOAD_SIZE}³")
print(f"  Dataset ID: {DATASET_ID}")
print(f"  Benchmark codecs: {len(BENCHMARK_CODECS)}")
print(f"  Output: {OUTPUT_DIR}")

## QC Analysis Class

Implements PixelPatrol-inspired quality control checks

In [ ]:
class CryoETQualityControl:
    """Quality Control for CryoET tomogram data"""

    def __init__(self, data, name="tomogram", thresholds=None):
        self.data = data
        self.name = name
        self.thresholds = thresholds or QC_THRESHOLDS
        self.report = {}

    def run_all_checks(self):
        """Run comprehensive QC analysis"""
        print(f"Running QC checks on {self.name}...")

        self.check_basic_stats()
        self.check_distribution()
        self.check_outliers()
        self.check_snr()
        self.check_dynamic_range()
        self.check_saturation()
        self.check_spatial_consistency()

        self.report["timestamp"] = datetime.now().isoformat()
        self.report["status"] = self.determine_status()

        return self.report

    def check_basic_stats(self):
        """Basic statistical properties"""
        self.report["shape"] = self.data.shape
        self.report["dtype"] = str(self.data.dtype)
        self.report["size_mb"] = self.data.nbytes / (1024**2)
        self.report["min"] = float(np.min(self.data))
        self.report["max"] = float(np.max(self.data))
        self.report["mean"] = float(np.mean(self.data))
        self.report["std"] = float(np.std(self.data))
        self.report["median"] = float(np.median(self.data))

    def check_distribution(self):
        """Analyze pixel value distribution"""
        # Histogram statistics
        hist, bin_edges = np.histogram(self.data, bins=100)
        self.report["histogram"] = {
            "counts": hist.tolist(),
            "bin_edges": bin_edges.tolist(),
        }

        # Distribution tests
        self.report["skewness"] = float(stats.skew(self.data.flatten()))
        self.report["kurtosis"] = float(stats.kurtosis(self.data.flatten()))

    def check_outliers(self):
        """Detect outlier pixels"""
        mean = self.report["mean"]
        std = self.report["std"]
        threshold = self.thresholds["outlier_std"]

        outliers = np.abs(self.data - mean) > (threshold * std)
        n_outliers = np.sum(outliers)

        self.report["outliers"] = {
            "count": int(n_outliers),
            "fraction": float(n_outliers / self.data.size),
            "threshold_std": threshold,
        }

    def check_snr(self):
        """Estimate signal-to-noise ratio"""
        # Simple SNR estimate: mean / std
        snr = abs(self.report["mean"]) / (self.report["std"] + 1e-10)

        self.report["snr"] = {
            "value": float(snr),
            "pass": snr >= self.thresholds["min_snr"],
        }

    def check_dynamic_range(self):
        """Check intensity dynamic range"""
        data_range = self.report["max"] - self.report["min"]

        self.report["dynamic_range"] = {
            "value": float(data_range),
            "pass": data_range >= self.thresholds["min_dynamic_range"],
        }

    def check_saturation(self):
        """Check for saturated pixels"""
        # Assume saturation near min/max for float data
        data_min, data_max = self.report["min"], self.report["max"]
        margin = (data_max - data_min) * 0.01

        saturated_low = np.sum(self.data <= (data_min + margin))
        saturated_high = np.sum(self.data >= (data_max - margin))
        total_saturated = saturated_low + saturated_high

        saturation_fraction = total_saturated / self.data.size

        self.report["saturation"] = {
            "low_count": int(saturated_low),
            "high_count": int(saturated_high),
            "fraction": float(saturation_fraction),
            "pass": saturation_fraction <= self.thresholds["max_saturation"],
        }

    def check_spatial_consistency(self):
        """Check for spatial anomalies"""
        # Analyze variance across z-slices
        slice_means = np.mean(self.data, axis=(1, 2))
        slice_stds = np.std(self.data, axis=(1, 2))

        self.report["spatial"] = {
            "slice_mean_std": float(np.std(slice_means)),
            "slice_std_std": float(np.std(slice_stds)),
            "mean_consistency": float(
                np.std(slice_means) / (abs(np.mean(slice_means)) + 1e-10)
            ),
        }

    def determine_status(self):
        """Overall QC status"""
        checks = [
            self.report["snr"]["pass"],
            self.report["dynamic_range"]["pass"],
            self.report["saturation"]["pass"],
            self.report["outliers"]["fraction"] < 0.05,  # <5% outliers
        ]

        if all(checks):
            return "PASS"
        elif sum(checks) >= len(checks) * 0.75:
            return "WARNING"
        else:
            return "FAIL"

    def visualize(self, save_path=None):
        """Create QC visualization dashboard"""
        fig = plt.figure(figsize=(16, 12))
        gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

        # Row 1: Orthogonal slices
        mid_z, mid_y, mid_x = [s // 2 for s in self.data.shape]

        ax1 = fig.add_subplot(gs[0, 0])
        im1 = ax1.imshow(self.data[mid_z], cmap="gray")
        ax1.set_title(f"XY Slice (Z={mid_z})")
        ax1.axis("off")
        plt.colorbar(im1, ax=ax1, fraction=0.046)

        ax2 = fig.add_subplot(gs[0, 1])
        im2 = ax2.imshow(self.data[:, mid_y, :], cmap="gray")
        ax2.set_title(f"XZ Slice (Y={mid_y})")
        ax2.axis("off")
        plt.colorbar(im2, ax=ax2, fraction=0.046)

        ax3 = fig.add_subplot(gs[0, 2])
        im3 = ax3.imshow(self.data[:, :, mid_x], cmap="gray")
        ax3.set_title(f"YZ Slice (X={mid_x})")
        ax3.axis("off")
        plt.colorbar(im3, ax=ax3, fraction=0.046)

        # Row 2: Distribution analysis
        ax4 = fig.add_subplot(gs[1, 0])
        ax4.hist(
            self.data.flatten(),
            bins=100,
            alpha=0.7,
            color="steelblue",
            edgecolor="black",
        )
        ax4.axvline(
            self.report["mean"],
            color="red",
            linestyle="--",
            linewidth=2,
            label=f"Mean: {self.report['mean']:.2f}",
        )
        ax4.axvline(
            self.report["median"],
            color="green",
            linestyle="--",
            linewidth=2,
            label=f"Median: {self.report['median']:.2f}",
        )
        ax4.set_xlabel("Intensity")
        ax4.set_ylabel("Frequency")
        ax4.set_title("Intensity Distribution")
        ax4.legend()
        ax4.grid(True, alpha=0.3)

        ax5 = fig.add_subplot(gs[1, 1])
        slice_means = np.mean(self.data, axis=(1, 2))
        ax5.plot(slice_means, linewidth=2, color="navy")
        ax5.set_xlabel("Z Slice")
        ax5.set_ylabel("Mean Intensity")
        ax5.set_title("Spatial Consistency (Z)")
        ax5.grid(True, alpha=0.3)

        ax6 = fig.add_subplot(gs[1, 2])
        slice_stds = np.std(self.data, axis=(1, 2))
        ax6.plot(slice_stds, linewidth=2, color="darkred")
        ax6.set_xlabel("Z Slice")
        ax6.set_ylabel("Std Dev")
        ax6.set_title("Noise Profile (Z)")
        ax6.grid(True, alpha=0.3)

        # Row 3: QC metrics
        ax7 = fig.add_subplot(gs[2, :])
        ax7.axis("off")

        qc_text = f"""QC REPORT: {self.name}
        
Status: {self.report["status"]}
Shape: {self.report["shape"]}
Size: {self.report["size_mb"]:.2f} MB

Intensity Statistics:
  Range: [{self.report["min"]:.3f}, {self.report["max"]:.3f}]
  Mean: {self.report["mean"]:.3f} ± {self.report["std"]:.3f}
  Median: {self.report["median"]:.3f}
  Skewness: {self.report["skewness"]:.3f}
  Kurtosis: {self.report["kurtosis"]:.3f}

Quality Metrics:
  SNR: {self.report["snr"]["value"]:.2f} {"✓" if self.report["snr"]["pass"] else "✗"}
  Dynamic Range: {self.report["dynamic_range"]["value"]:.2f} {"✓" if self.report["dynamic_range"]["pass"] else "✗"}
  Saturation: {self.report["saturation"]["fraction"] * 100:.2f}% {"✓" if self.report["saturation"]["pass"] else "✗"}
  Outliers: {self.report["outliers"]["fraction"] * 100:.2f}% ({self.report["outliers"]["count"]} pixels)

Spatial Consistency:
  Mean variation: {self.report["spatial"]["mean_consistency"]:.4f}
  Slice std deviation: {self.report["spatial"]["slice_std_std"]:.3f}
        """

        ax7.text(
            0.05,
            0.95,
            qc_text,
            transform=ax7.transAxes,
            fontsize=10,
            verticalalignment="top",
            family="monospace",
            bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.5),
        )

        fig.suptitle(
            f"Quality Control Dashboard: {self.name}", fontsize=16, fontweight="bold"
        )

        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches="tight")

        plt.show()


print("✓ QC class defined")

## 1. Download CryoET Data

In [ ]:
print("Connecting to CryoET Data Portal...")
client = Client()
dataset = Dataset.get_by_id(client, DATASET_ID)

print(f"✓ Dataset: {dataset.title}")

# Find tomogram
runs = list(dataset.runs)
selected_tomo = None
for run in runs:
    for tomo in list(run.tomograms):
        if (
            tomo.size_x >= DOWNLOAD_SIZE
            and tomo.size_y >= DOWNLOAD_SIZE
            and tomo.size_z >= DOWNLOAD_SIZE
        ):
            selected_tomo = tomo
            break
    if selected_tomo:
        break

print(f"✓ Selected: {selected_tomo.name}")

# Download data
s3 = s3fs.S3FileSystem(anon=True)
zarr_path = selected_tomo.s3_omezarr_dir.replace("s3://", "")
store = s3fs.S3Map(root=zarr_path, s3=s3, check=False)
zarr_group = zarr.open(store, mode="r")
zarr_array = zarr_group["0"]

# Download centered cube
actual_size = min(DOWNLOAD_SIZE, min(zarr_array.shape))
z_c = zarr_array.shape[0] // 2
y_c = zarr_array.shape[1] // 2
x_c = zarr_array.shape[2] // 2

z_start = max(0, z_c - actual_size // 2)
z_end = min(zarr_array.shape[0], z_start + actual_size)
y_start = max(0, y_c - actual_size // 2)
y_end = min(zarr_array.shape[1], y_start + actual_size)
x_start = max(0, x_c - actual_size // 2)
x_end = min(zarr_array.shape[2], x_start + actual_size)

print(f"\nDownloading {actual_size}³ cube...")
reference_data = np.array(zarr_array[z_start:z_end, y_start:y_end, x_start:x_end])

print(f"✓ Downloaded: {reference_data.shape}")
print(f"  Size: {reference_data.nbytes / (1024**2):.2f} MB")

## 2. Pre-Compression Quality Control

Validate data quality before compression testing

In [ ]:
print("=" * 80)
print("PRE-COMPRESSION QUALITY CONTROL")
print("=" * 80)

# Run QC on original data
qc_original = CryoETQualityControl(reference_data, name="Original Data")
pre_qc_report = qc_original.run_all_checks()

print(f"\n✓ QC Status: {pre_qc_report['status']}")

# Visualize
qc_original.visualize(save_path=OUTPUT_DIR / "qc_original.png")

# Save report
with open(OUTPUT_DIR / "qc_pre_compression.json", "w") as f:
    json.dump(pre_qc_report, f, indent=2)

print("\n✓ Pre-compression QC complete")
print(f"  Report saved to: {OUTPUT_DIR / 'qc_pre_compression.json'}")

## 3. Compression Benchmarking with QC

Test compression while monitoring data integrity

In [ ]:
print("=" * 80)
print("COMPRESSION BENCHMARKING WITH QC")
print("=" * 80)

benchmark_results = []
qc_results = {}

for idx, (codec, shuffle, level) in enumerate(BENCHMARK_CODECS):
    print(
        f"\n[{idx + 1}/{len(BENCHMARK_CODECS)}] Testing: {codec} (shuffle={shuffle}, level={level})"
    )

    # Create unique name
    config_name = f"{codec}_{shuffle}_{level}" if shuffle else f"{codec}_L{level}"
    store_path = OUTPUT_DIR / f"{config_name}.zarr"
    utils.remove_output_dir(store_path)

    # Get compressor
    chunks = (CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE)

    if "blosc" in codec:
        cname = codec.split("_")[1]
        compressor = read_write_zarr.get_blosc_compressor(cname, level, shuffle)
    elif codec == "zstd":
        compressor = read_write_zarr.get_zstd_compressor(level)
    else:
        compressor = read_write_zarr.get_gzip_compressor(level)

    # Benchmark write
    t0 = time.time()
    read_write_zarr.write_zarr_array(
        reference_data,
        store_path,
        overwrite=False,
        chunks=chunks,
        compressor=compressor,
        zarr_spec=ZARR_SPEC,
    )
    write_time = time.time() - t0

    # Benchmark read
    t0 = time.time()
    read_back = read_write_zarr.read_zarr_array(store_path)
    read_time = time.time() - t0

    # Get metrics
    ratio = read_write_zarr.get_compression_ratio(store_path)
    size_mb = utils.get_directory_size(store_path) / (1024**2)

    # Quality metrics
    ssim_val = ssim(
        (reference_data[reference_data.shape[0] // 2] - reference_data.min())
        / (reference_data.max() - reference_data.min() + 1e-10),
        (read_back[read_back.shape[0] // 2] - read_back.min())
        / (read_back.max() - read_back.min() + 1e-10),
        data_range=1.0,
    )

    data_range = reference_data.max() - reference_data.min()
    psnr_val = psnr(reference_data, read_back, data_range=data_range)
    mse_val = mse(reference_data, read_back)

    # Post-compression QC
    print("  Running post-compression QC...")
    qc_compressed = CryoETQualityControl(read_back, name=config_name)
    qc_report = qc_compressed.run_all_checks()
    qc_results[config_name] = qc_report

    # Store results
    benchmark_results.append(
        {
            "config": config_name,
            "codec": codec,
            "shuffle": shuffle or "N/A",
            "level": level,
            "write_time": write_time,
            "read_time": read_time,
            "ratio": ratio,
            "size_mb": size_mb,
            "ssim": ssim_val,
            "psnr": psnr_val,
            "mse": mse_val,
            "qc_status": qc_report["status"],
            "qc_snr": qc_report["snr"]["value"],
            "data_identical": np.array_equal(reference_data, read_back),
        }
    )

    print(f"  Write: {write_time:.3f}s, Read: {read_time:.3f}s")
    print(f"  Ratio: {ratio:.3f}×, Size: {size_mb:.2f} MB")
    print(f"  QC Status: {qc_report['status']}, SSIM: {ssim_val:.6f}")
    print(f"  Data identical: {np.array_equal(reference_data, read_back)}")

print("\n✓ Benchmark complete")

## 4. QC Comparison: Before vs After Compression

In [ ]:
# Create comparison DataFrame
df_bench = pd.DataFrame(benchmark_results)

# Save results
df_bench.to_csv(OUTPUT_DIR / "benchmark_with_qc.csv", index=False)

print("=" * 80)
print("QC COMPARISON: ORIGINAL vs COMPRESSED")
print("=" * 80)

print("\nOriginal Data QC:")
print(f"  Status: {pre_qc_report['status']}")
print(f"  SNR: {pre_qc_report['snr']['value']:.2f}")
print(f"  Dynamic Range: {pre_qc_report['dynamic_range']['value']:.2f}")
print(f"  Outliers: {pre_qc_report['outliers']['fraction'] * 100:.2f}%")

print("\nCompressed Data QC:")
for config in df_bench["config"]:
    qc_rep = qc_results[config]
    print(
        f"  {config:30s}: Status={qc_rep['status']}, SNR={qc_rep['snr']['value']:.2f}"
    )

# Check if all compressions preserved data
all_identical = df_bench["data_identical"].all()
print(f"\n✓ All compressions are lossless: {all_identical}")
print(f"✓ All QC checks passed: {(df_bench['qc_status'] == 'PASS').all()}")

## 5. Integrated QC + Benchmark Dashboard

In [ ]:
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Plot 1: Compression Ratio vs Time
ax1 = fig.add_subplot(gs[0, 0])
colors = ["green" if s == "PASS" else "orange" for s in df_bench["qc_status"]]
ax1.scatter(
    df_bench["write_time"],
    df_bench["ratio"],
    c=colors,
    s=200,
    alpha=0.6,
    edgecolors="black",
)
for idx, row in df_bench.iterrows():
    ax1.annotate(
        row["config"],
        (row["write_time"], row["ratio"]),
        fontsize=8,
        ha="center",
        va="bottom",
    )
ax1.set_xlabel("Write Time (s)")
ax1.set_ylabel("Compression Ratio")
ax1.set_title("Compression vs Speed (color=QC status)")
ax1.grid(True, alpha=0.3)

# Plot 2: SSIM Quality
ax2 = fig.add_subplot(gs[0, 1])
ax2.bar(range(len(df_bench)), df_bench["ssim"], color="steelblue", alpha=0.7)
ax2.set_xticks(range(len(df_bench)))
ax2.set_xticklabels(df_bench["config"], rotation=45, ha="right", fontsize=8)
ax2.set_ylabel("SSIM")
ax2.set_title("Image Quality (SSIM)")
ax2.axhline(y=1.0, color="red", linestyle="--", label="Perfect")
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Storage Savings
ax3 = fig.add_subplot(gs[0, 2])
savings = (1 - df_bench["size_mb"] / pre_qc_report["size_mb"]) * 100
ax3.bar(range(len(df_bench)), savings, color="green", alpha=0.7)
ax3.set_xticks(range(len(df_bench)))
ax3.set_xticklabels(df_bench["config"], rotation=45, ha="right", fontsize=8)
ax3.set_ylabel("Storage Savings (%)")
ax3.set_title("Storage Reduction")
ax3.grid(True, alpha=0.3)

# Plot 4: SNR Comparison
ax4 = fig.add_subplot(gs[1, 0])
snr_values = [pre_qc_report["snr"]["value"]] + df_bench["qc_snr"].tolist()
labels = ["Original"] + df_bench["config"].tolist()
ax4.bar(range(len(snr_values)), snr_values, alpha=0.7)
ax4.set_xticks(range(len(snr_values)))
ax4.set_xticklabels(labels, rotation=45, ha="right", fontsize=8)
ax4.set_ylabel("SNR")
ax4.set_title("Signal-to-Noise Ratio")
ax4.axhline(y=QC_THRESHOLDS["min_snr"], color="red", linestyle="--", label="Threshold")
ax4.legend()
ax4.grid(True, alpha=0.3)

# Plot 5: Performance Matrix
ax5 = fig.add_subplot(gs[1, 1])
perf_data = df_bench[["write_time", "read_time"]].values.T
im = ax5.imshow(perf_data, cmap="RdYlGn_r", aspect="auto")
ax5.set_yticks([0, 1])
ax5.set_yticklabels(["Write", "Read"])
ax5.set_xticks(range(len(df_bench)))
ax5.set_xticklabels(df_bench["config"], rotation=45, ha="right", fontsize=8)
ax5.set_title("Performance Heat Map")
for i in range(len(df_bench)):
    for j in range(2):
        ax5.text(i, j, f"{perf_data[j, i]:.3f}", ha="center", va="center", fontsize=8)
plt.colorbar(im, ax=ax5, label="Time (s)")

# Plot 6: QC Status Summary
ax6 = fig.add_subplot(gs[1, 2])
qc_status_counts = df_bench["qc_status"].value_counts()
colors_pie = [
    "green" if x == "PASS" else "orange" if x == "WARNING" else "red"
    for x in qc_status_counts.index
]
ax6.pie(
    qc_status_counts.values,
    labels=qc_status_counts.index,
    autopct="%1.0f%%",
    colors=colors_pie,
    startangle=90,
)
ax6.set_title("QC Status Distribution")

# Plot 7: Summary Table
ax7 = fig.add_subplot(gs[2, :])
ax7.axis("off")

summary_text = f"""INTEGRATED QC + BENCHMARK SUMMARY

Dataset: {selected_tomo.name} ({reference_data.shape})
Pre-compression QC: {pre_qc_report["status"]}

Benchmark Results ({len(df_bench)} configurations tested):

{"Config":<30} {"Ratio":<8} {"Write(s)":<10} {"Read(s)":<10} {"SSIM":<10} {"QC Status":<12}
{"-" * 90}
"""

for _, row in df_bench.iterrows():
    summary_text += f"{row['config']:<30} {row['ratio']:<8.3f} {row['write_time']:<10.3f} {row['read_time']:<10.3f} {row['ssim']:<10.6f} {row['qc_status']:<12}\n"

summary_text += f"""\n\nRECOMMENDATION:
Best overall: {df_bench.loc[df_bench["ratio"].idxmax(), "config"]} (ratio={df_bench["ratio"].max():.3f}×)
Fastest: {df_bench.loc[df_bench["write_time"].idxmin(), "config"]} (write={df_bench["write_time"].min():.3f}s)
All compressions are lossless: {all_identical}
"""

ax7.text(
    0.05,
    0.95,
    summary_text,
    transform=ax7.transAxes,
    fontsize=9,
    verticalalignment="top",
    family="monospace",
    bbox=dict(boxstyle="round", facecolor="lightblue", alpha=0.5),
)

fig.suptitle(
    "CryoET Quality Control + Benchmarking Dashboard", fontsize=18, fontweight="bold"
)

plt.savefig(OUTPUT_DIR / "qc_benchmark_dashboard.png", dpi=150, bbox_inches="tight")
plt.show()

print(f"\n✓ Dashboard saved to: {OUTPUT_DIR / 'qc_benchmark_dashboard.png'}")

## 6. Generate Comprehensive Report

In [ ]:
# Create comprehensive report
report = {
    "metadata": {
        "timestamp": datetime.now().isoformat(),
        "dataset_id": DATASET_ID,
        "tomogram": selected_tomo.name,
        "data_shape": reference_data.shape,
        "download_size_mb": reference_data.nbytes / (1024**2),
    },
    "pre_qc": pre_qc_report,
    "benchmark_summary": {
        "configurations_tested": len(df_bench),
        "all_lossless": all_identical,
        "all_qc_passed": (df_bench["qc_status"] == "PASS").all(),
        "best_compression": {
            "config": df_bench.loc[df_bench["ratio"].idxmax(), "config"],
            "ratio": float(df_bench["ratio"].max()),
            "savings_mb": float(
                pre_qc_report["size_mb"]
                - df_bench.loc[df_bench["ratio"].idxmax(), "size_mb"]
            ),
        },
        "fastest": {
            "config": df_bench.loc[df_bench["write_time"].idxmin(), "config"],
            "write_time": float(df_bench["write_time"].min()),
        },
    },
    "post_qc": qc_results,
    "benchmark_results": df_bench.to_dict("records"),
}

# Save comprehensive report
report_path = OUTPUT_DIR / "comprehensive_qc_benchmark_report.json"
with open(report_path, "w") as f:
    json.dump(report, f, indent=2)

print("=" * 80)
print("COMPREHENSIVE QC + BENCHMARK REPORT")
print("=" * 80)
print("\n✓ Analysis complete")
print("\nFiles generated:")
print(f"  - {report_path}")
print(f"  - {OUTPUT_DIR / 'benchmark_with_qc.csv'}")
print(f"  - {OUTPUT_DIR / 'qc_original.png'}")
print(f"  - {OUTPUT_DIR / 'qc_benchmark_dashboard.png'}")
print("\nRecommendations:")
print(
    f"  Best compression: {report['benchmark_summary']['best_compression']['config']}"
)
print(
    f"    → Saves {report['benchmark_summary']['best_compression']['savings_mb']:.2f} MB"
)
print(f"  Fastest: {report['benchmark_summary']['fastest']['config']}")
print(f"    → Write time: {report['benchmark_summary']['fastest']['write_time']:.3f}s")
print(
    f"\n✓ All compressions are lossless: {report['benchmark_summary']['all_lossless']}"
)
print(f"✓ All QC checks passed: {report['benchmark_summary']['all_qc_passed']}")

## Next Steps

### Extend QC Checks:
- Add domain-specific quality metrics
- Integrate with PixelPatrol for advanced profiling
- Custom outlier detection algorithms

### Automate Testing:
- Batch process multiple datasets
- CI/CD integration for regression testing
- Automated alerts for QC failures

### Advanced Analysis:
- Compare QC metrics across datasets
- Machine learning for QC prediction
- Correlation analysis: QC ↔ compression performance

### Resources:
- PixelPatrol: https://github.com/ida-mdc/pixel-patrol
- CryoET Portal: https://cryoetdataportal.czscience.com/
- Zarr Benchmarks: https://github.com/HEFTIEProject/zarr-benchmarks